# Graphs and Graph Algorithms

In [8]:
class Node:
    
    def __init__(self, value):
        self.data = value
        self.color = None
        self.explored = False
        self.num_neighbors = 0
        self.neighbors = {}
    
    def remove_neighbor(self, n):
        if n in self.neighbors:
            self.neighbors.pop(n)
            self.num_neighbors -= 1
    
    def add_neighbor(self, n):
        if not n.data in self.neighbors:
            self.neighbors[n.data] = n
            self.num_neighbors += 1
        
    def show_neighbors(self):
        print "        neighbors: " + str([n.data for n in self.neighbors.values()])
    
    def show(self):
        print "Node %s: " % self.data
        print "        color: " + str( self.color )
        print "        explored: " + str(self.explored)
        print "        neighbors: " + str([n.data for n in self.neighbors.values()])
        print ""


class Graph:
    
    def __init__(self, edges, directed = False):
        self.directed = directed
        self.vertices = {}
        
        for e in set(edges):
            self.add_edge(e)
    
    def add_edge(self, e):
        self.add_vertex(e[0], neighbors = [e[1]])
    
    def add_vertex(self, v, neighbors = []):
        if not v in self.vertices:
            self.vertices[v] = Node(v)
        for n in neighbors:
            self.add_vertex(n)
            self.vertices[v].add_neighbor(self.vertices[n])
            if not self.directed:
                self.vertices[n].add_neighbor(self.vertices[v])
    
    def delete_edge(self,e):
        if e[1] in self.vertices[e[0]].neighbors:
            self.vertices[e[0]].remove_neighbor(e[1])
        if not self.directed:
            if e[0] in self.vertices[e[1]].neighbors:
                self.vertices[e[1]].remove_neighbor(e[0])
    
    def delete_vertex(self, del_val):
        if del_val in self.vertices:
            del_node = self.vertices.pop(del_val) # remove from vertex list
        if self.directed:
            # O(|V|) for a directed graph
            for v, n in self.vertices.items():
                n.remove_neighbor(del_val)
        else:
            # O(|V_i|) for an undirected graph, where i is the degree of the node being deleted
            for v, n in del_node.neighbors.items():
                n.remove_neighbor(del_val)
    
    def show(self):
        print "directed: %s" % str(self.directed)
        for v in self.vertices.values():
            v.show()

In [9]:
edges = [("A","B"),("A","D"),("B","C"),("C","D")]

print ""
print "Undirected: "
print ""
graph = Graph(edges)
graph.show()

print ""
print ""

print "Directed: "
print ""
graph = Graph(edges, True)
graph.show()


Undirected: 

directed: False
Node A: 
        color: None
        explored: False
        neighbors: ['B', 'D']

Node C: 
        color: None
        explored: False
        neighbors: ['B', 'D']

Node B: 
        color: None
        explored: False
        neighbors: ['A', 'C']

Node D: 
        color: None
        explored: False
        neighbors: ['A', 'C']



Directed: 

directed: True
Node A: 
        color: None
        explored: False
        neighbors: ['B', 'D']

Node C: 
        color: None
        explored: False
        neighbors: ['D']

Node B: 
        color: None
        explored: False
        neighbors: ['C']

Node D: 
        color: None
        explored: False
        neighbors: []



In [10]:
edges = [("A","B"),("A","D"),("B","C"),("C","D")]
graph = Graph(edges, True)
graph.show()
print ""
print "Delete 'd': "
graph.delete_vertex("D")
graph.show()

directed: True
Node A: 
        color: None
        explored: False
        neighbors: ['B', 'D']

Node C: 
        color: None
        explored: False
        neighbors: ['D']

Node B: 
        color: None
        explored: False
        neighbors: ['C']

Node D: 
        color: None
        explored: False
        neighbors: []


Delete 'd': 
directed: True
Node A: 
        color: None
        explored: False
        neighbors: ['B']

Node C: 
        color: None
        explored: False
        neighbors: []

Node B: 
        color: None
        explored: False
        neighbors: ['C']



## Bipartite Graph

In [11]:
def is_bypartite_graph(n):
    explored = set()
    explored.add(n.data)
    def _is_bypartite_graph_rec(n):
        print "Exploring: " + str(n.data)
        for u in n.neighbors.values():
            if u.color == n.color: # we found the same colors next to each other
                return False
            if u.color == None:
                u.color = (not n.color)
        
        to_explore = set(n.neighbors.keys()).difference(explored)
        if len(to_explore) < 1:
            return True
        else:
            res = True
            for u in to_explore:
                explored.add(u)
                res = res and _is_bypartite_graph_rec(n.neighbors[u])
            return res
    # call rec
    return _is_bypartite_graph_rec(n)

        
edges = [("A","B"),("A","D"),("B","C"),("C","D")]
graph = Graph(edges)
start = graph.vertices.values()[0]
start.color = False
print "Graph 1 is a bipartite graph: " + str(is_bypartite_graph(start) == True)
print ""

edges =  [("A","C"),("A","D"),("B","C"),("C","D")]
graph2 = Graph(edges)
start2 = graph2.vertices.values()[0]
start2.color = False
print "Graph 2 is a bipartite graph: " + str(is_bypartite_graph(start2) == True)

Exploring: A
Exploring: B
Exploring: C
Exploring: D
Exploring: D
Graph 1 is a bipartite graph: True

Exploring: A
Exploring: C
Graph 2 is a bipartite graph: False


## Path finding

### There exists a path from a to b
In a given grid (graph), find if there exists a path from a starting point to an end point (0,0 -> 5,5) 

In [12]:
grid = [[0, 0, 0, 0, 0, 1],
        [1, 1, 0, 0, 0, 1],
        [0, 0, 0, 1, 0, 0],
        [0, 1, 0, 1, 0, 1],
        [0, 1, 0, 0, 1, 0],
        [0, 1, 0, 0, 0, 2]]


def search_rec(x, y):
    if grid[y][x] == 2:
        print 'found end at %d,%d' % (x, y)
        return True
    elif grid[y][x] == 1:
        print 'wall at %d,%d' % (x, y)
        return False
    elif grid[y][x] == 3:
        print 'visited at %d,%d' % (x, y)
        return False
    
    print 'visiting %d,%d' % (x, y)

    # mark as visited
    grid[y][x] = 3

    # explore neighbors clockwise starting by the one on the right
    if (
        (x < len(grid[0])-1 and search_rec(x+1, y))  # right
        or (y < len(grid)-1 and search_rec(x, y+1)) # down
        or (x > 0 and search_rec(x-1, y))        # search left
        or (y > 0 and search_rec(x, y-1))        # search up
        ):
        return True

    return False

search_rec(0, 0)

visiting 0,0
visiting 1,0
visiting 2,0
visiting 3,0
visiting 4,0
wall at 5,0
visiting 4,1
wall at 5,1
visiting 4,2
visiting 5,2
wall at 5,3
visited at 4,2
wall at 5,1
visiting 4,3
wall at 5,3
wall at 4,4
wall at 3,3
visited at 4,2
wall at 3,2
visited at 4,1
visiting 3,1
visited at 4,1
wall at 3,2
visiting 2,1
visited at 3,1
visiting 2,2
wall at 3,2
visiting 2,3
wall at 3,3
visiting 2,4
visiting 3,4
wall at 4,4
visiting 3,5
visiting 4,5
found at 5,5


True

### Depth First Traversal

In [30]:
def deapth_first_traversal(start_node, prefix = True, func = None):
    if func == None:
        def print_value(node):
            print node.data
        func = print_value
    
    def dfs_rec(this_node):
        this_node.explored = True
        
        if prefix:
            func(this_node)
        
        for n in this_node.neighbors.values():
            if not n.explored:
                dfs_rec(n)
           
        if not prefix:
            func(this_node)
    
    dfs_rec(start_node)

In [34]:
edges = [("A","B"),("A","D"),("B","C"),("C","D"),("D","E"),("B","F"),("F","G")]
graph = Graph(edges)
start_node = graph.vertices.values()[0]
deapth_first_traversal(start_node)

A
B
C
D
E
F
G


### Breadth First Traversal

In [35]:

def breadth_first_traversal(start_node, func = None):
    if func == None:
        def print_value(node):
            print node.data
        func = print_value
    
    def breadth_rec(this_node):
        this_node.explored = True
        
        for n in this_node.neighbors.values():
            if not n.color:
                n.color = "done"
                func(n)
        
        for n in this_node.neighbors.values():
            if not n.explored:
                breadth_rec(n)
    
    func(start_node)
    start_node.color = "done"
    breadth_rec(start_node)

In [36]:
edges = [("A","B"),("A","D"),("B","C"),("C","D"),("D","E"),("B","F"),("F","G")]
graph = Graph(edges)
start_node = graph.vertices.values()[0]
breadth_first_traversal(start_node)

from sets import Set
from collections import namedtuple



A
B
D
C
F
E
G


### A* search

Impliment the A* search algorithm to find a path from a to b